In [1]:
import os
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.metrics import pairwise_distances
from scipy.stats import pearsonr, spearmanr, f_oneway
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import r2_score, accuracy_score

# ==================================================
# CONFIGURATION – CHANGE THIS FOR EACH EXPERIMENT
# ==================================================
model_name = "qnn_q14_l12_b64_acc2_s42_20260228_115810_best"   # your saved model name
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Analyzing experiment: {model_name}")

# Load the saved data
embeddings = np.load(f"{model_name}_embeddings.npz")
train_emb = embeddings['train_emb']
val_emb   = embeddings['val_emb']
test_emb  = embeddings['test_emb']

# Combine all embeddings (full dataset)
quantum_embeddings = np.vstack([train_emb, val_emb, test_emb])
print(f"Quantum embeddings shape: {quantum_embeddings.shape}")

# Load predictions and true values
preds = np.load(f"{model_name}_predictions.npz")
train_pred = preds['train_pred']
train_true = preds['train_true']
val_pred   = preds['val_pred']
val_true   = preds['val_true']
test_pred  = preds['test_pred']
test_true  = preds['test_true']

# Load hyperparameters (optional)
with open(f"{model_name}_params.txt", 'r') as f:
    params = f.read()
print("Hyperparameters:\n", params)

Analyzing experiment: qnn_q14_l12_b64_acc2_s42_20260228_115810_best


FileNotFoundError: [Errno 2] No such file or directory: 'qnn_q14_l12_b64_acc2_s42_20260228_115810_best_embeddings.npz'

In [ ]:
# Load the original dataframe (same as in training)
DRIVE_URL = "https://drive.google.com/uc?id=1PS0eB8dx8VMzVvxNUc6wBzsMRkEKJjWI"
df = pd.read_csv(DRIVE_URL)

# You'll need the same scaler and feature list if you want to map indices
# But for now, we need the reaction names and V_B, etc.
# Load the reaction split (to know test/val/train indices)
OUTDIR_BASE = "mdn_70_10_20_optimized"
train_reacts = pd.read_csv(f"{OUTDIR_BASE}/train_reactions.csv")["Reaction"].values
val_reacts   = pd.read_csv(f"{OUTDIR_BASE}/val_reactions.csv")["Reaction"].values
test_reacts  = pd.read_csv(f"{OUTDIR_BASE}/test_reactions.csv")["Reaction"].values

train_mask = df["Reaction"].isin(train_reacts)
val_mask   = df["Reaction"].isin(val_reacts)
test_mask  = df["Reaction"].isin(test_reacts)

# For full dataset indices, we can use the mask to split later if needed

# Load classical MDN ensemble data (for switch and regime labels)
ENSEMBLE_DIR = os.path.join(OUTDIR_BASE, "ensembles_fast")
seed_dirs = sorted([os.path.join(ENSEMBLE_DIR, d) for d in os.listdir(ENSEMBLE_DIR) if d.startswith("seed_")])
all_seed_components = []
for seed_path in seed_dirs:
    npz_path = os.path.join(seed_path, "mdn_all_components.npz")
    if os.path.exists(npz_path):
        data = np.load(npz_path)
        all_seed_components.append({"pi": data["pi"]})
print("Loaded classical MDN seeds:", len(all_seed_components))

# Also load one seed for regime labels (e.g., seed 42)
seed_path = os.path.join(ENSEMBLE_DIR, "seed_42", "mdn_all_components.npz")
data_mdn = np.load(seed_path)
pi_mdn = data_mdn["pi"]
dominant_regime = np.argmax(pi_mdn, axis=1)
print("Dominant regime shape:", dominant_regime.shape)

In [ ]:
# Attach embeddings to dataframe (full dataset)
df_emb = df.copy()
for i in range(quantum_embeddings.shape[1]):
    df_emb[f"q_{i}"] = quantum_embeddings[:, i]

# Compute reaction-level mean embedding
reaction_emb = df_emb.groupby("Reaction")[[f"q_{i}" for i in range(quantum_embeddings.shape[1])]].mean().reset_index()
print("Reaction-level embedding shape:", reaction_emb.shape)

# Load reliable switch from classical MDN (reuse your earlier code)
def compute_switch_per_seed(df, pi_array):
    # (same as your function)
    df_temp = df.copy().reset_index(drop=True)
    df_temp["dominant"] = np.argmax(pi_array, axis=1)
    switch_dict = {}
    for reaction, sub in df_temp.groupby("Reaction"):
        sub = sub.sort_values("E c.m.").reset_index(drop=True)
        dom = sub["dominant"].values
        E_vals = sub["E c.m."].values
        if len(dom) < 2:
            continue
        switch_energy = np.nan
        for i in range(1, len(dom)):
            if dom[i] != dom[i-1]:
                switch_energy = E_vals[i]
                break
        if not np.isnan(switch_energy):
            V_B = sub["V_B"].iloc[0]
            x_switch = switch_energy / V_B
            switch_dict[reaction] = x_switch
    return switch_dict

seed_switch_results = [compute_switch_per_seed(df, seed["pi"]) for seed in all_seed_components]

all_reactions = df["Reaction"].unique()
switch_records = []
for reaction in all_reactions:
    x_list = [sd[reaction] for sd in seed_switch_results if reaction in sd]
    if len(x_list) > 0:
        switch_records.append({
            "Reaction": reaction,
            "x_switch_mean": np.mean(x_list),
            "x_switch_std": np.std(x_list),
            "seed_fraction": len(x_list)/len(seed_switch_results)
        })
switch_df = pd.DataFrame(switch_records)
switch_df_clean = switch_df[switch_df["seed_fraction"] >= 0.8].copy()
print("Reliable switches:", len(switch_df_clean))

# Merge with reaction embeddings
merged = reaction_emb.merge(switch_df_clean[["Reaction","x_switch_mean"]], on="Reaction", how="inner")
print("Merged shape:", merged.shape)

# Quantum matrix
Q_mat = merged[[f"q_{i}" for i in range(quantum_embeddings.shape[1])]].values
x_vals = merged["x_switch_mean"].values

# PCA on reaction-level embeddings
pca = PCA(n_components=3)
Q_pca = pca.fit_transform(Q_mat)
print("Explained variance ratio:", pca.explained_variance_ratio_)

# Correlate PC1 with x_switch
pc1 = Q_pca[:,0]
corr, pval = pearsonr(pc1, x_vals)
print(f"PC1 vs x_switch: r={corr:.3f}, p={pval:.3e}")

# Plot
plt.figure(figsize=(6,5))
plt.scatter(pc1, x_vals, alpha=0.7)
plt.xlabel("Quantum PC1")
plt.ylabel("x_switch")
plt.title(f"{model_name}\nPC1 vs x_switch (r={corr:.3f})")
plt.grid(alpha=0.3)
plt.savefig(f"{model_name}_pc1_vs_switch.png", dpi=150)
plt.show()

In [ ]:
# Merge structural parameters
struct_df = df.groupby("Reaction").first().reset_index()
merged = merged.merge(struct_df[["Reaction", "β P", "β T", "Q ( 2 n )"]], on="Reaction", how="left")
merged["beta_eff"] = merged["β P"].abs() + merged["β T"].abs()

beta_vals = merged["beta_eff"].values
Q2n_vals = merged["Q ( 2 n )"].values

print("PC1 vs beta_eff:", pearsonr(pc1, beta_vals))
print("PC1 vs Q(2n):", pearsonr(pc1, Q2n_vals))

# Linear models
X_pcs = Q_pca[:, :3]
lr_pcs = LinearRegression().fit(X_pcs, x_vals)
print("R2 (3 PCs → x_switch):", r2_score(x_vals, lr_pcs.predict(X_pcs)))

X_struct = np.column_stack([beta_vals, Q2n_vals])
lr_struct = LinearRegression().fit(X_struct, x_vals)
print("R2 (β_eff+Q2n → x_switch):", r2_score(x_vals, lr_struct.predict(X_struct)))

In [ ]:
# Use the full quantum embeddings and the MDN regime labels (from seed 42)
pca_sample = PCA(n_components=3)
Q_pca_sample = pca_sample.fit_transform(quantum_embeddings)
print("Explained variance ratio:", pca_sample.explained_variance_ratio_)

pca_df = pd.DataFrame({
    "PC1": Q_pca_sample[:, 0],
    "PC2": Q_pca_sample[:, 1],
    "Regime": dominant_regime
})

plt.figure(figsize=(8,6))
for regime in np.unique(dominant_regime):
    subset = pca_df[pca_df["Regime"] == regime]
    plt.scatter(subset["PC1"], subset["PC2"], alpha=0.4, label=f"Regime {regime}")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.title(f"{model_name}\nQuantum PCA Colored by MDN Regime")
plt.legend()
plt.grid(alpha=0.3)
plt.savefig(f"{model_name}_pca_regimes.png", dpi=150)
plt.show()

# Classification accuracy using PC1 only
clf = LogisticRegression(max_iter=1000)
clf.fit(Q_pca_sample[:, 0].reshape(-1,1), dominant_regime)
acc = accuracy_score(dominant_regime, clf.predict(Q_pca_sample[:, 0].reshape(-1,1)))
print(f"Regime classification accuracy (PC1 only): {acc:.3f}")

# ANOVA
groups = [Q_pca_sample[dominant_regime == r, 0] for r in np.unique(dominant_regime)]
f_stat, p_val = f_oneway(*groups)
print(f"ANOVA PC1 across regimes: F={f_stat:.1f}, p={p_val:.3e}")

In [ ]:
# Pairwise quantum distances vs |Δ x_switch|
# (You can reuse your earlier code, but now at reaction level)
# This may be redundant with PC1 correlation, but included for completeness.

In [ ]:
# Save merged reaction data with PCs and structural params
merged.to_csv(f"{model_name}_reaction_analysis.csv", index=False)

# Save sample PCA data
np.savez(f"{model_name}_sample_pca.npz",
         pc_scores=Q_pca_sample,
         explained_ratio=pca_sample.explained_variance_ratio_)

print("All analysis results saved.")